In [10]:
from bs4 import BeautifulSoup
import requests
import datetime
import time
import pandas as pd
import json
import re
from pprint import pprint
import csv
import pickle

In [12]:
kiva = pd.read_csv('/Users/travis/Downloads/kiva_ds_csv/loans.csv')
#kiva = pd.read_json('/Users/travis/Downloads/kiva_ds_json/loans.json')

In [13]:
kiva.head()

,LOAN_ID,LOAN_NAME,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,FUNDED_AMOUNT,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,...,LENDER_TERM,NUM_LENDERS_TOTAL,NUM_JOURNAL_ENTRIES,NUM_BULK_ENTRIES,TAGS,BORROWER_NAMES,BORROWER_GENDERS,BORROWER_PICTURED,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,1511246.0,NaN,...,7.0,3,2,1,NaN,Aivy,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,1507947.0,NaN,...,8.0,11,2,1,NaN,Idalia Marizza,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,1512245.0,NaN,...,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",Aasia,female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,1514419.0,NaN,...,14.0,21,2,1,user_favorite,Gulmira,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,1506830.0,NaN,...,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",Ricky\t,male,true,bullet,field_partner


In [14]:
kiva.drop(['ORIGINAL_LANGUAGE', 'DESCRIPTION', 'DESCRIPTION_TRANSLATED',
           'IMAGE_ID', 'VIDEO_ID', 'TOWN_NAME', 'TAGS', 'BORROWER_NAMES',
           'BORROWER_PICTURED', 'LOAN_USE', 'COUNTRY_CODE'],
          axis=1,
          inplace=True)

In [15]:
kiva.describe()

,LOAN_ID,FUNDED_AMOUNT,LOAN_AMOUNT,CURRENCY_EXCHANGE_COVERAGE_RATE,PARTNER_ID,LENDER_TERM,NUM_LENDERS_TOTAL,NUM_JOURNAL_ENTRIES,NUM_BULK_ENTRIES
count,1.419607e+06,1.419607e+06,1.419607e+06,1.098081e+06,1.402817e+06,1.419583e+06,1.419607e+06,1.419607e+06,1.419607e+06
mean,7.233713e+05,7.961254e+02,8.322284e+02,1.163657e-01,1.496207e+02,1.305139e+01,2.225389e+01,1.502054e+00,1.134976e+00
std,4.156766e+05,1.034257e+03,1.080551e+03,3.699645e-02,8.769345e+01,7.566660e+00,2.777410e+01,9.903614e-01,4.950988e-01
min,8.400000e+01,0.000000e+00,2.500000e+01,1.000000e-01,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,3.642165e+05,2.750000e+02,3.000000e+02,1.000000e-01,9.800000e+01,8.000000e+00,8.000000e+00,1.000000e+00,1.000000e+00
50%,7.240350e+05,5.000000e+02,5.000000e+02,1.000000e-01,1.390000e+02,1.200000e+01,1.500000e+01,1.000000e+00,1.000000e+00
75%,1.082972e+06,9.500000e+02,1.000000e+03,1.000000e-01,1.740000e+02,1.400000e+01,2.700000e+01,2.000000e+00,1.000000e+00
max,1.444085e+06,1.000000e+05,1.000000e+05,2.000000e-01,5.570000e+02,1.950000e+02,3.045000e+03,4.800000e+01,2.400000e+01


In [16]:
kiva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419607 entries, 0 to 1419606
Data columns (total 23 columns):
LOAN_ID                            1419607 non-null int64
LOAN_NAME                          1372049 non-null object
FUNDED_AMOUNT                      1419607 non-null float64
LOAN_AMOUNT                        1419607 non-null float64
STATUS                             1419607 non-null object
ACTIVITY_NAME                      1419607 non-null object
SECTOR_NAME                        1419607 non-null object
COUNTRY_NAME                       1419607 non-null object
CURRENCY_POLICY                    1419607 non-null object
CURRENCY_EXCHANGE_COVERAGE_RATE    1098081 non-null float64
CURRENCY                           1419607 non-null object
PARTNER_ID                         1402817 non-null float64
POSTED_TIME                        1419607 non-null object
PLANNED_EXPIRATION_TIME            1047773 non-null object
DISBURSE_TIME                      1416794 non-null object

In [ ]:
kiva[(kiva['LOAN_AMOUNT'] > kiva['FUNDED_AMOUNT']) & (kiva['DISTRIBUTION_MODEL'] == 'direct')]

In [ ]:
kiva['COUNTRY_NAME'].value_counts()

In [ ]:
f, ax = plt.subplots(figsize=(12,12))
sns.countplot(x='COUNTRY_NAME', data=kiva)

In [17]:
kiva.to_pickle('kiva.pkl')

In [18]:
#kiva = pd.read_pickle('kiva.pkl')

In [20]:
kiva2 = kiva.sample(50000)

In [22]:
kiva2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 96810 to 65664
Data columns (total 23 columns):
LOAN_ID                            50000 non-null int64
LOAN_NAME                          48308 non-null object
FUNDED_AMOUNT                      50000 non-null float64
LOAN_AMOUNT                        50000 non-null float64
STATUS                             50000 non-null object
ACTIVITY_NAME                      50000 non-null object
SECTOR_NAME                        50000 non-null object
COUNTRY_NAME                       50000 non-null object
CURRENCY_POLICY                    50000 non-null object
CURRENCY_EXCHANGE_COVERAGE_RATE    38772 non-null float64
CURRENCY                           50000 non-null object
PARTNER_ID                         49383 non-null float64
POSTED_TIME                        50000 non-null object
PLANNED_EXPIRATION_TIME            36869 non-null object
DISBURSE_TIME                      49896 non-null object
RAISED_TIME                  

In [24]:
kiva2.head()

,LOAN_ID,LOAN_NAME,FUNDED_AMOUNT,LOAN_AMOUNT,STATUS,ACTIVITY_NAME,SECTOR_NAME,COUNTRY_NAME,CURRENCY_POLICY,CURRENCY_EXCHANGE_COVERAGE_RATE,...,PLANNED_EXPIRATION_TIME,DISBURSE_TIME,RAISED_TIME,LENDER_TERM,NUM_LENDERS_TOTAL,NUM_JOURNAL_ENTRIES,NUM_BULK_ENTRIES,BORROWER_GENDERS,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL
96810,486224,ROSEL,250.0,250.0,funded,Fruits & Vegetables,Food,Philippines,shared,0.1,...,2012-12-02 01:40:02.000 +0000,2012-10-10 07:00:00.000 +0000,2012-11-06 22:54:12.000 +0000,7.0,7,2,1,female,irregular,field_partner
606022,292769,Omar,1225.0,1225.0,funded,Farming,Agriculture,Georgia,shared,0.2,...,NaN,2011-04-08 07:00:00.000 +0000,2011-04-25 17:03:09.000 +0000,14.0,39,1,1,male,monthly,field_partner
694977,27002,Aurora Mateo,600.0,600.0,funded,Food Production/Sales,Food,Dominican Republic,not shared,NaN,...,NaN,2007-12-25 18:11:49.000 +0000,2007-12-11 18:11:49.000 +0000,8.0,18,1,1,female,monthly,field_partner
1099612,550132,Shermuhammad,850.0,850.0,funded,Cattle,Agriculture,Tajikistan,shared,0.1,...,2013-05-22 12:50:02.000 +0000,2013-04-04 07:00:00.000 +0000,2013-04-22 21:22:31.000 +0000,10.0,32,2,1,male,monthly,field_partner
930983,922644,Ansar,300.0,300.0,funded,Animal Sales,Agriculture,Pakistan,shared,0.1,...,2015-08-28 14:10:05.000 +0000,2015-07-08 07:00:00.000 +0000,2015-08-11 09:31:39.000 +0000,14.0,9,2,1,female,monthly,field_partner


In [30]:
kiva2.STATUS.value_counts()

funded         47545
expired         2086
refunded         240
fundRaising      129
Name: STATUS, dtype: int64